# Quantization on MNIST data set, 2d attempt 

## Working as expected 

#### Imports

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

#### Data

In [24]:
# Using tf MNIST data set this time 
mnist = tf.keras.datasets.mnist
fash = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0

#### 2nd Model

In [29]:
model2 = tf.keras.Sequential([
  tf.keras.layers.InputLayer(input_shape=(28, 28)),
  tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
  tf.keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10)
])

# Train the digit classification model
model2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])
model2.fit(
  train_images,
  train_labels,
  epochs=5,
  validation_data=(test_images, test_labels)
)
eval = model2.evaluate(test_images, test_labels)
print("test loss, test acc:", eval)
tf.saved_model.save(model2, './model2_mnist/')

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.4902 - accuracy: 0.8305 - val_loss: 0.3908 - val_accuracy: 0.8618
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3575 - accuracy: 0.8758 - val_loss: 0.3548 - val_accuracy: 0.8771
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3246 - accuracy: 0.8859 - val_loss: 0.3370 - val_accuracy: 0.8812
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2998 - accuracy: 0.8955 - val_loss: 0.3151 - val_accuracy: 0.8886
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.3034 - accuracy: 0.8913
[0.3034186363220215, 0.8913000226020813]
INFO:tensorflow:Assets written to: ./model2_mnist/assets


INFO:tensorflow:Assets written to: ./model2_mnist/assets


## 1. Quantization blueprint

In [80]:
path = './model1_mnist/'
converter = tf.lite.TFLiteConverter.from_saved_model(path)

In [81]:
# Conversion set-up 

converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops.
]

converter.optimizations = [tf.lite.Optimize.DEFAULT]

model = converter.convert()

### No quantization

In [82]:
# Converted model interpreter 

interpreter = tf.lite.Interpreter(model_content=model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print('Input details : ', input_details)
print('Output details :', output_details)

Input details :  [{'name': 'serving_default_flatten_input:0', 'index': 0, 'shape': array([  1, 784], dtype=int32), 'shape_signature': array([ -1, 784], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details : [{'name': 'StatefulPartitionedCall:0', 'index': 12, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


Since the input shape is set to 1,784 ( from the original TF model) we resize the input tensor before allocating it again.   
Then we can load the whole test set to produce a production result that we can interpret thanks to argmax.

In [83]:
test_images = test_images.reshape(10000,784)
interpreter.resize_tensor_input(input_index = input_details[0]['index'],tensor_size=(10000,784))
interpreter.allocate_tensors()
interpreter.set_tensor(input_details[0]['index'], test_images)

interpreter.invoke()
tflite_predictions = interpreter.get_tensor(output_details[0]['index'])

Here we get 98% prediction, which would be the precision o the noraml model, but no quantization was actually done : the datatype is till float32

In [84]:
pred = np.argmax(tflite_predictions, axis=1)
print('Accuracy : ', (pred == test_labels).mean() )

Accuracy :  0.0707


## 2. Post-training integer quantization

Now we will reproduce the steps, but with the addition of the quantization steps

In [85]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(1000):
    yield [input_value]

converter = tf.lite.TFLiteConverter.from_saved_model(path)

converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS_INT8#,  
    # tf.lite.OpsSet.TFLITE_BUILTINS,
    # tf.lite.OpsSet.SELECT_TF_OPS  
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

quant_model = converter.convert()

interpreter = tf.lite.Interpreter(model_content=quant_model)

In [86]:
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>


In [87]:
# To replace in the interpreter.set_tensor line for single inference : 

# test_images[10] = np.expand_dims(test_images[10], axis=0).astype(input_type)
# test_images[10] = test_images[10] / input_scale + input_zero_point
# dici = test_images[10].astype(input_type)

In [88]:
# Interpreter
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_scale, input_zero_point = input_details[0]["quantization"]

#whole set 
interpreter.resize_tensor_input(input_index = input_details[0]['index'],tensor_size=(10000,784))
interpreter.allocate_tensors()

test_images_q = test_images / input_scale + input_zero_point
test_images_q = np.reshape(test_images_q.astype(input_type), (10000,784))

interpreter.set_tensor(input_details[0]['index'], test_images_q)

# Invoke
interpreter.invoke()
tflite_predictions2 = interpreter.get_tensor(output_details[0]['index'])
#tflite_predictions2

In [89]:
pred2 = np.argmax(tflite_predictions2, axis=1)
print('Accuracy : ', (pred2 == test_labels).mean())
print('Predictions similarity noquant vs. quant : ', (pred2 == pred).mean())

Accuracy :  0.07
Predictions similarity noquant vs. quant :  0.9926


In [90]:
# import matplotlib.pyplot as plt
# #plt.imshow(np.reshape(test_images_int8[97], (28,28)))
# plt.show()
# plt.imshow(np.reshape(test_images[10], (28,28)))
# type(test_images)

## Automatization

#### Data

In [17]:
mnist = tf.keras.datasets.mnist
fash = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fash.load_data()

# Very important step
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0

#### Model 
Here I can load any presaved model or train a new one

In [20]:
# Careful on what dataset model2 (and 1) is trained
saved_model1 = './model1_mnist/'
saved_model2 = './model2_mnist/'

#### Functions

In [21]:
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(1000):
        yield [input_value]

def conv(model_path):
    # TODO set option to read it from saved model or from existing model
    converter1 = tf.lite.TFLiteConverter.from_saved_model(model_path)
    converter1.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter1.optimizations = [tf.lite.Optimize.DEFAULT]
    converter1.representative_dataset = representative_data_gen
    converter1.inference_input_type = tf.uint8
    converter1.inference_output_type = tf.uint8

    quant_model = converter1.convert()
    return(quant_model)


def interpret(model, test_set):
    interpreter = tf.lite.Interpreter(model_content=model)

    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    input_type = interpreter.get_input_details()[0]['dtype']
    # Quantization parameters : 
    input_scale, input_zero_point = input_details[0]["quantization"]

    # whole set, currently only supports test_set as an array
    interpreter.resize_tensor_input(input_index=input_details[0]['index'], tensor_size=np.shape(test_set))
    interpreter.allocate_tensors()

    # 8bit quantization approximation
    test_images_q = test_set / input_scale + input_zero_point
    test_images_q = np.reshape(test_images_q.astype(input_type), np.shape(test_set)) # too complex

    # Loading into the tensor
    interpreter.set_tensor(input_details[0]['index'], test_images_q)

    return interpreter

def run_inference(interpreter):
    output_details = interpreter.get_output_details()
    interpreter.invoke()
    inference = interpreter.get_tensor(output_details[0]['index'])
    predictions = np.argmax(inference, axis=1)
    print('Accuracy : ', (predictions == test_labels).mean())

    return predictions

#### Execution 

In [30]:
# possible TODO : make it only in a single function, with (saved_model, test_images) params
quanted_model = conv(saved_model2)
interpreter = interpret(quanted_model, test_images)
run_inference(interpreter)

Accuracy :  0.8923


array([9, 2, 1, ..., 8, 1, 5])

With model2 :
on mnist   : accuracy 0.98
on fashion : accuracy 0.89